# Modelisation

In [ ]:
from __future__ import division
import sys
import os

# Dataframe manipulation
import pandas as pd
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 50) # to see more lines
pd.options.mode.chained_assignment = None  # default='warn'

import pickle

# General
import numpy as np
import math
from collections import Counter
import multiprocessing
from multiprocessing import Pool
import time
from tqdm import tqdm
from itertools import groupby
from scipy.stats import pearsonr

# Text mining
    # cleaning
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer, PorterStemmer
    # tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
from emoji import UNICODE_EMOJI
from nltk import tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Graphs
import networkx as nx

# Plots
import matplotlib.pyplot as plt

# URLs analysis
import re
import urllib.parse
from urllib.parse import urlparse
import math

# for exportation
import csv

from datetime import datetime

# sklearn
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid

from parfit import bestFit

# Boosting algorithms
import xgboost as xgb

# NN
from tensorflow import keras
from keras import optimizers, regularizers, callbacks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.utils.vis_utils import model_to_dot

In [ ]:
# to read
#os.chdir("C:\\Users\Jade\Documents\COURS\M2\S2\WebMining\Project")
with open('df_v13.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
# merging with 25 text mining features (we had to create them appart)
with open('token_features.pkl', 'rb') as f:
    token_features = pickle.load(f)

token_features.set_index('id', inplace=True, drop=True)

In [ ]:
df.set_index('id', inplace=True, drop=True)
df = df.merge(token_features, on='id')

In [ ]:
df.to_pickle("df_all_features.pkl")

In [ ]:
# to read
#os.chdir("C:\\Users\Jade\Documents\COURS\M2\S2\WebMining\Project")
with open('df_all_features.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
token_names = list(token_features.columns)
token_names.remove("time")
token_names.append("time_y")
del token_features

Let's split the variables into three groups

In [ ]:
numerical_features = ['nb_linked_sr', 'subjectivity', 'nb_urls', 'nb_pop_urls', 
                      'word_count', 'exclamation', 'vocabulary_richness', 'nb_emojis',
                      'nb_author_thread', 'nb_neighbours', 'depth',
                      'degree_centrality', 'betweenness_centrality', 'eigenvector_centrality', 
                      'order_com', 'parent_score', 'time_lapse', 'time_since_parent', 'nb_com_author']

boolean_features = ['is_root', 'is_quoting']
boolean_features.extend(token_names)

categorical_features = ['hour', 'weekday']

all_features = []
all_features.extend(categorical_features)
all_features.extend(numerical_features)
all_features.extend(boolean_features)

print("We have", len(numerical_features), "numerical features,", 
      len(boolean_features), "boolean features and",
      len(categorical_features), "categorical features.")

The name of the variable **is_root** could be misleading. It just corresponds to comments that are a reply to the initial question in the Askreddit, they are in the first layer.

In [ ]:
df[numerical_features].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

In [ ]:
variance_too_large = [i for i in numerical_features if df[i].std() > 5]
variance_too_large

Some variables such as word_count, have a much larger variance than others, we need to be careful with that! We will scale these features in the data preparation step below.

# Models
To submit the Kaggle we will need to predict on test, using what we have learned on train, so we will need the test to have the same columns as test (except for the ups).

## Checking for correlation

Before including our features into our model, we need to check if a variable is "highly" correlated with the target variable. Indeed, such variables do not necessarily improve the model so adding them would be at the expense of efficiency (higher dimensionality).

Let's check the correlation between each numerical feature, including the boolean (coded by 0 and 1) with ups. 
What we usually mean by a "high" correlation is when the Pearson correlation coefficient between 2 numerical variables is above 0.7. 

In [ ]:
# Selection of features
var_select = []
var_select.extend(numerical_features)
var_select.extend(boolean_features)
var_select.append('ups')

# Pearson correlation coefficient
df_corr = df[var_select]
matrix_corr = df_corr.corr(method ='pearson') 
corr_num_ups = matrix_corr['ups'][:-1]
corr_num_ups

There is no coefficient above 0.7, hence we can keep them all without a risk of anormally high correlation with the target.

## Data preparation

In [ ]:
# Dropping unnecessary columns for prediction
useless_cols = ["created_utc", "link_id", "name", "author", "body", "parent_id", "time_x"]
df.drop(columns=useless_cols, inplace=True)

In [ ]:
# Checking column names
print(df.columns, '\n')

print("Our column names are fine, they don't contain brackets or weird symbols.")

#### Encoding categorical variables
The goal here is to transform categorical variables into numerical so that they are suitable to be used in machine learning techniques. After some research, I realized there are many ways to encode, adapted to different kinds of issues such as high cardinality, ordered or nominal, etc. <br>
In our case we don't have any variable with a lot of modalities (more than 1000) so we don't need to worry about high cardinality issues, but we still have some categorical feautres that seem really important to determine the score, so we must still be careful if we want to do it right. <br>
The thing is that many articles in the litterature have shown that the type of encoding that we use should depend on the type of model that we intend to implement. For instance, some boosting models (such as CatBoost) are able to operate with categorical features directly, whereas neural networks or linear models require these features to be previously transformed to a numerical version (one dummy for every modality, for instance). Besides, for a tree-based model, **some papers suggest that a single level of a one-hot-encoded categorical variable must have a very high importance score (which is measured by how much it decreases tree impurity) in order to be selected for splitting at an early stage (i.e. closer to the root), over a continuous variable**. And so, because of the fact that continuous variables would be more likely to be selected from the beggining, it could happen that the subsequent branches are impure and that even if we include categorical features at a later stage, it is already too late for obtaining an optimal performance....

In our case we only have 2 categorical features (4 if we count the boolean ones), so we won't implement catboost. Hence, taking all of this into account, in what follows we will:
- ordinally encode multinomial **hierarchical** categorical features, to keep the hierarchy of modalities: hour and weekday (we don't need to do anything because it's already ordinally encoded)
- one-hot encode simple ones (non-hierarchical ones): leave booleans as integers

##### Scaling
This is a necessary step. From sklearn-learn.org: "If a feature has a variance that is orders of magnitude larger that others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected."
Hence we will standard scale every feature with too large variance.

In [ ]:
# Selecting what we are going to scale
df_to_scale = df[variance_too_large]

# Keeping all the rest
features_to_append = [i for i in all_features if i not in variance_too_large]
features_to_append.append('ups')
to_append = df[features_to_append]

In [ ]:
# Standard scaling (remove mean and scale to unit variance)
scaler = StandardScaler()

scaled_np_array = StandardScaler().fit_transform(df_to_scale.values)
df_scaled = pd.DataFrame(scaled_np_array, index=df_to_scale.index, columns=df_to_scale.columns)

In [ ]:
df_scaled = pd.concat([df_scaled, to_append], axis=1)

##### Splitting into X, y and X_test

In [ ]:
mask = df_scaled.ups.isnull()
X = df_scaled[~mask]
y = X.ups
X = X.drop('ups', axis=1)

test = df_scaled[mask]
X_test = test.drop('ups', axis=1) # Kaggle submission will be predictions on X_test

In [ ]:
# Splitting into train and validation sets
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
y_validation.to_frame().describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

### Useful functions

In [ ]:
def model_evaluation(model):
    """Evaluates model performances and tells if we overfit or not, and by how much"""
    
    predicted_train = model.predict(X_train.values)
    predicted_validation = model.predict(X_validation.values)
    
    train_MAE = round(mean_absolute_error(y_train, predicted_train), 4)
    validation_MAE = round(mean_absolute_error(y_validation, predicted_validation), 4)
    
    print("Training MAE:", train_MAE)
    print("Validation MAE:", validation_MAE)
    if train_MAE < validation_MAE:
        print("We overfit...")
    else:
        print("We don't overfit!")
        
    return((train_MAE, validation_MAE))

In [ ]:
def create_csv(kaggle_prediction):
    """Returns a csv files with predictions in the good format for Kaggle submission"""
    csvData = [['id', 'Predicted']]
    indices = X_test.index.to_list()
    print("Creating csv...")
    for i, j in zip(range(len(indices)), range(len(kaggle_prediction))):
        csvData.append([indices[i], kaggle_prediction[j]])
    
    filename = 'submission_' + datetime.today().strftime('%m_%d_%H_%M') + '.csv'
    with open(filename, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerows(csvData)
    csvFile.close()

In [ ]:
def get_feature_importances(model):
    ft_weights_xgb_reg = pd.DataFrame(model.feature_importances_, columns=['weight'], index=X_train.columns)
    ft_weights_xgb_reg.sort_values('weight', inplace=True)
    plt.figure(figsize=(8, 10))
    plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg.weight, align='center') 
    plt.title(f"Feature importances in the {model.__class__} model", fontsize=14)
    plt.margins(y=0.01)
    plt.show()

In [ ]:
dico_model_performances = {} # we'll fill this dico progressively

## XGBoost
For the past recent years, XGBoost has been the king algorithm delivering fast and high performance for regression problems.

### First run 

In [ ]:
# Model
xgb_reg = xgb.XGBRegressor(n_jobs=-1, random_state=1139, verbosity=2, max_depth=5)
xgb_reg.fit(X_train.values, y_train.values)

In [ ]:
# Prediction results
dico_model_performances['xgb_reg'] = {model_evaluation(xgb_reg)}

We overfit too much, let's run some parameter tuning

In [ ]:
get_feature_importances(xgb_reg)

nb_neighbours is by far the most important feature! It's quite impressive how the difference is so huge

### Simple parameter tunning using $k$-fold CV ($k$=5)

General parameters:
* booster (default: gbtree): type of model. Default gbtree performs always better than gblinear.
* seed: used for reproducible results, useful for parameter tunning.
* nthread: nb of parallel threads used.

Parameters we will tune:
* n_estimators: number of trees to fit
* max_depth: maximum depth of a tree. Will be used to control over-fitting (higher depth => more specific tree).

Other parameters redundant with what we already used (max_leaf_nodes, colsample_bylevel) or for which default value is okay (gamma, max_delta_step, etc.) won't be included.

In [ ]:
params = {
    "n_estimators": np.array([10, 60, 120]),
    "max_depth": np.array([4,5,6])
}

# We start by fixing the rest to standard values
xgb1 = xgb.XGBRegressor(
    eta = 0.1,
    njobs = -1,
    seed=1139)

In [ ]:
best_model, best_score, all_models, all_scores = bestFit(
    model = xgb1,
    paramGrid = ParameterGrid(params), 
    X_train = X_train,
    y_train = y_train,
    X_val = X_validation,
    y_val = y_validation,
    metric = mean_absolute_error,
    # Choice between optimizing for greater scores or lesser scores Default True means greater and False means lesser
    scoreLabel = "mean_absolute_error",
    greater_is_better = False)

### XGBoost last run: optimized parameters

Here we will replace n_estimators and max_depth with the best parameters.

To avoid overfitting, we will also decrease subsample (fraction of observations to be randomly sampled for each tree).

In [ ]:
opt_n_estimators = 60
opt_max_depth = 5

In [ ]:
##### Model
xgb_reg_star = xgb.XGBRegressor(n_estimators=opt_n_estimators,
                                eta=0.1,
                                max_depth=opt_max_depth,
                                subsample=0.9,
                                colsample_bytree=0.9,
                                gamma=0,
                                n_jobs=-1,
                                verbosity=2,
                                seed=1139)

xgb_reg_star.fit(X_train.values, y_train.values)

In [ ]:
# Prediction results
dico_model_performances['xgb_reg_star'] = {model_evaluation(xgb_reg_star)}

In [ ]:
get_feature_importances(xgb_reg_star)

In [ ]:
# Kaggle prediction
kaggle_prediction = np.round(xgb_reg_star.predict(X_test.values), 1)
create_csv(kaggle_prediction)

## Neural Network

In [ ]:
### Building a simple NN
nn = Sequential()
nn.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
nn.add(Dense(256, activation='relu'))
nn.add(Dense(256, activation='relu'))
nn.add(Dense(1, activation='linear'))

# Compiling the model
nn.compile(loss='mean_absolute_error',
            optimizer="adam",
            metrics=['mean_absolute_error'])

# Model summary
print(nn.summary())

In [ ]:
# Training the model
early_stopping = callbacks.EarlyStopping(monitor='mean_absolute_error', patience=7, verbose=1)
# When the model doesn't improve after 15 iterations, we stop even if it hasn't finished (saves us time).

nn_history = nn.fit(X_train.values,
                    y_train.values,
                    batch_size=256,
                    epochs=2,
                    verbose=1,
                    callbacks=[early_stopping],
                    use_multiprocessing=True)

In [ ]:
dico_model_performances['nn_base'] = {model_evaluation(nn)}

In [ ]:
# Kaggle prediction
a = np.round(nn.predict(X_test.values), 1)
kaggle_prediction = np.array([a[i][0] for i in range(len(a))])
create_csv(kaggle_prediction)

We tunned manually. Below is the best model along with the results from experimental trials.

## Experimental results: (25 epochs)

1. 4 layers (128, 256, 256, 1) / no regularization / no batchnormalization / default adam / batch_size=256 => (train_MAE=8.0477, validation_MAE=8.1296) Kaggle: 9.29011
* 4 layers (128, 256, 256, 1) / dropout=0.2 / no batchnormalization / default adam / batch_size=256 => (train_MAE=7.9864, validation_MAE=8.054) Kaggle: 9.50644
* 3 layers (128, 256, 1) / dropout=0.2 / no batchnormalization / custom adam / batch_size=256 => (train_MAE=8.1594, validation_RMSE=8.2105) Kaggle: 9.54065,
* **4 layers (128, 256, 256, 1) / dropout=0.4 / batchnormalization / custom adam / batch_size=256 => (train_MAE=23.0333, validation_MAE=27.939)**

The best model was the first one, but Neural Network is no better than XGBoost.

# Models Comparison

In [ ]:
all_performances = [dico_model_performances[key][1] for key in dico_model_performances.keys()]
all_performances = sorted(all_performances)

fig, ax = plt.subplots(figsize = (12,6))

ax.bar(dico_model_performances.keys(), all_performances, color="green")
plt.title('Models Comparison')
plt.xlabel('Models')
plt.xticks(rotation = 45)
plt.ylabel('Mean Absolute Error')